# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
# import libraries
# !pip install git+http://github.com/scikit-learn/scikit-learn.git

# !pip install git+http://github.com/scikit-learn/scikit-multilearn.git

import sqlalchemy as sql
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
# from skmultilearn.problem_transform import BinaryRelevance

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
# from sklearn.metrics import multilabel_confusion_matrix

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
# load data from database
engine = sql.create_engine('sqlite:///MessageCategory.db')
conn = engine.connect()
df = pd.read_sql_table('MessageCategory', conn)
df.dropna(axis='index',inplace=True)
def CovertBtoUtf8(x):
    if isinstance(x, bytes):
        return x.decode('utf-8')
    else:
        return x

df.message = df.message.apply(CovertBtoUtf8)
X = pd.DataFrame(df.message)
# X = pd.DataFrame(df['message'].apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x))
# Y = df[['medical_products']]
Y = df[['related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report']]


In [5]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10153 entries, 0 to 12333
Data columns (total 1 columns):
message    10153 non-null object
dtypes: object(1)
memory usage: 158.6+ KB


In [7]:
df.head(1)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
X.head(1)['message']

0    Weather update - a cold front from Cuba that c...
Name: message, dtype: object

### 2. Write a tokenization function to process your text data

In [9]:
def tokenize(text):
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    stop_words = stopwords.words("english")
    tokens = [t for t in tokens if t not in stop_words]
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [11]:
def returnMessage(X):
    print(X)
    return X

In [10]:
# Build customer function estimator for message Genre`
# Direct messages has more weight as compared to Social or News

class MessageGenreWeight(BaseEstimator, TransformerMixin):

    def assign_genre_weight(self, genre):
              
        genre_weight = {
            'direct' : int(10),
            'social' : int(5),
            'news'   : int(2)
        }
        
        return genre_weight.get(genre,0)

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_genre_weight = X['message'].apply(self.assign_genre_weight) 
        return pd.DataFrame(X_genre_weight)

In [12]:
mgw = MessageGenreWeight()
mgw.transform(X).head(2)

,message
0,0
1,0


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
#Build model pipline 

# 1. CounterVectoriser + TfidfTransformer in parallel to MessageGenreWeight customer estimator
# 2. Randomforest Classifier using MultiOutputClassifier

# rfc = RandomForestClassifier()

# pipeline =  Pipeline([
#                 ('features',FeatureUnion([
#                     
#                     ('text_pipeline',Pipeline([
#                         ('vect', CountVectorizer(tokenizer=tokenize)),
#                         ('tfidf', TfidfTransformer())
#                     ])),
#                     
#                     ('MessageGenreWeight', MessageGenreWeight())
#                     
#                 ])),
#     
#                 ('MultiLabelClf', MultiOutputClassifier(estimator=rfc))
#                 ('rfc',  RandomForestClassifier())
#             ])

In [13]:
rfc = RandomForestClassifier(n_jobs=100)

pipeline = Pipeline([
                ('vect',CountVectorizer(tokenizer=tokenize)),
                ('tfidf',TfidfTransformer()),
#                 ('clf', OneVsRestClassifier(estimator=rfc, n_jobs=Y_test.shape[1]))
    
                ('clf', MultiOutputClassifier(estimator=rfc))
#                 ('rfc',RandomForestClassifier())
    
            ])

In [ ]:
# features = FeatureUnion([
#                 ('text_pipeline',Pipeline([
#                     ('vect',CountVectorizer(tokenizer=tokenize,preprocessor=returnMessage)),
#                     ('tfidf',TfidfTransformer())
#                 ])),
#                 ('MessageGenreWeight',MessageGenreWeight())
#             ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [53]:
def display_results(cv, y_test, y_pred, labels):
    
#      confusion_mat = multilabel_confusion_matrix(y_test, y_pred, labels=labels, samplewise=True)
    confusion_mat = confusion_matrix(y_test.values, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy: \n", accuracy)
    print("\nModel):", cv)

In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [ ]:
# !pip install pixiedust
# import pixiedust

In [16]:
Y_train.shape

(7614, 36)

In [20]:
X_train.values.ravel().shape

(7614,)

In [55]:
# %%pixie_debugger
# model = pipeline.fit(X_train.values.ravel(),  Y_train.values)
model = pipeline.fit(X_train, Y_train)

Y_pred = model.predict(X_test)
# display_results(model, Y_test.values.ravel(), Y_pred.ravel())

ValueError: Found input variables with inconsistent numbers of samples: [1, 7614]

In [48]:
Y_test.shape

(2539, 36)

In [22]:
Y_pred.shape

(1, 36)

In [49]:
print(model.predict(X_test))

[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


In [25]:
# for x in X_test.values:
#     y_pred = model.predict(x.item())
#     print(classification_report(Y_test[x.index], y_pred, target_names = Y_test.columns))

In [54]:
# display_results(model, Y_test.values, Y_pred)
display_results(model, Y_test, Y_pred, Y_test.columns)

ValueError: Found input variables with inconsistent numbers of samples: [2539, 1]

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [45]:
from sklearn.metrics import f1_score

for y in Y_test:
   print(y, y.index())
#    print(f1_score(Y_test.,Y_pred))

# Y_pred = model.predict(x)
# print(classification_report(Y_test, y_pred, target_names = Y_test.columns))

TypeError: index() takes at least 1 argument (0 given)

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.